# Walls to thermal circuit

In [1]:
import pandas as pd
import pd_dm4bem

## Wall types

Let's consider two types of walls described in the file `wall_types.csv`.

In [2]:
folder_path = "bldg_wall2TC"

wall_types = pd.read_csv(folder_path + '/wall_types.csv')
wall_types

,type,Material,Conductivity,Specific heat,Density,Width,Mesh
0,0,Concrete,1.400,880,2300,0.200,1
1,0,Insulation,0.027,1210,55,0.080,2
2,1,Glass,1.400,750,2500,0.004,1


The walls of __type__ 0 have two layers: Concrete and Insulation. Their physical characteristics (conductivity, specific heat, density) are given in SI units (W·m⁻¹·K⁻¹, J·kg⁻¹·K⁻¹, and kg·m⁻³, respectively). The width of the layers is in m. __Mesh__ represents the number of meshes in which the layer is discretized. Each mesh has two resistances and a capacity.

The wall __owO__ (in red) in Figures 1 and 2 is of type 0. It has two layers (concrete and insulation), each one discretized in one mesh.

## Walls

There are three kinds of walls:
- generic,
- outdoor,
- indoor.

The `walls_generic.csv`, `walls_out.csv`, and `walls_in.csv` files contain the characteristics of the surfaces of the walls (Table 1).

>Table 1. Columns of `walls_generic.csv`, `walls_out.csv`, and `walls_in.csv` files.

|Column|Unit|Definition                             | Example|_generic|_out|_in|
|------|----|---------------------------------------|--------|--------|---|---|
| ID   |    | Wall identifiant                      | w2     | x      | x | x |
| type |    | Type of wall given in `wall_types.csv`| 1      | x      | x | x |
| Area | m² | Surface area of the walll             | 25     | x      | x | x |
| β    | °  | Tilt angle: 90° vertical; >90° downward facing| 90 | x  | x |   |
| γ    | °  | Azimuth: 0° South, 180° North, >0 westward | 90| x      | x |   |
| T0   | °C | Temperature source of outer surface   | To     | x      | x |   |
| T1   | °C | Temperature source of outer surface   | Ti     | x      |   |   |
| Q0   | W  | Flow rate source on outer surface     | Qo     | x      | x | x |
| Q1   | W  | Flow rate source on inner surface     | Qi     | x      | x | x |
| h0   | W·m⁻²·K⁻¹| Convection coefficient of outer surface | 25 | x  | x | x |
| h1   | W·m⁻²·K⁻¹| Convection coefficient of inner surface |  8 | x  | x | x |
| α0   |    | Short-wave absorbtion coefficient of outer surface | 0.25 | x | x | x |
| α1   |    | Short-wave absorbtion coefficient of inner surface | 0.30 | x | x | x |
| ε0   |    | Long-wave hemispherical emissivity of outer surface| 0.85 | x | x | x |
| ε1   |    | Long-wave hemispherical emissivity of inner surface| 0.70 | x | x | x |
| y    |    | Output temperature nodes by using slicing |[0, -1] | x  | x | x |

__Notes__:
1. The _generic_ walls may characterized _out_ or _in_ walls by not specifying temperare sources (see `wall_generic.csv`, `wall_out.csv`, and `wall_in.csv` files for examples).

2. The _out_ walls do not need the temperature sources on the inner surfaces, _T1_.

3. The _in_ walls do not need tilt, β, and azimuth, γ, angles and the temperature sources on the outer, _T0_, and inner, _T1_, surfaces.

### Generic wall

Generic walls contain all the information listed in Table 1. For example, `wall_generic;csv` describes the walls shown in Figure 1.

Outdoor walls, such as gw0 and gw2 in Figure 1, can be obtained from the generic walls by not putting a temperature source T1.

Likewise, indoor walls,, such as gw3 in figure 1, can be obained from the genreic walls by not indicating temperature sources in T0 and T1. 

![generic_wall](./bldg_wall2TC/walls_generic.svg)
> Figure 1. Four examples of generic walls described in file `wall_generic.csv`.

In [3]:
walls = pd.read_csv(folder_path + "/walls_generic.csv")
walls

,ID,type,Area,β,γ,T0,T1,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,90,0,To,NaN,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,90,0,To,Ti,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,0,90,To,NaN,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,0,90,NaN,NaN,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


In [4]:
# Thermal circuits from data & type files of walls
TCd_generic = pd_dm4bem.wall2TC(wall_types, walls, prefix="g")

print('TCd_generic')
for key in TCd_generic.keys():
    print('Wall:', key)
    pd_dm4bem.print_TC(TCd_generic[key])

TCd_generic
Wall: gw0
A:
        gw0_θ0  gw0_θ1  gw0_θ2  gw0_θ3
gw0_q0     1.0     0.0     0.0     0.0
gw0_q1    -1.0     1.0     0.0     0.0
gw0_q2     0.0    -1.0     1.0     0.0
gw0_q3     0.0     0.0    -1.0     1.0 

G:
gw0_q0     20.0
gw0_q1    700.0
gw0_q2    700.0
gw0_q3     10.0
dtype: float64 

C:
gw0_θ0       0.0
gw0_θ1    7500.0
gw0_θ2       0.0
gw0_θ3       0.0
dtype: float64 

b:
gw0_q0     To
gw0_q1    0.0
gw0_q2    0.0
gw0_q3    0.0
dtype: object 

f:
gw0_θ0     Qo
gw0_θ1    0.0
gw0_θ2     Qi
gw0_θ3    0.0
dtype: object 

y:
gw0_θ0    1.0
gw0_θ1    0.0
gw0_θ2    0.0
gw0_θ3    1.0
dtype: float64 

Wall: gw1
A:
        gw1_θ0  gw1_θ1  gw1_θ2
gw1_q0     1.0     0.0     0.0
gw1_q1    -1.0     1.0     0.0
gw1_q2     0.0    -1.0     1.0
gw1_q3     0.0     0.0    -1.0 

G:
gw1_q0     20.0
gw1_q1    700.0
gw1_q2    700.0
gw1_q3     10.0
dtype: float64 

C:
gw1_θ0       0.0
gw1_θ1    7500.0
gw1_θ2       0.0
dtype: float64 

b:
gw1_q0     To
gw1_q1    0.0
gw1_q2    0.0
gw1_q3    

### Outdoor walls

Outdoor walls do not have temperature sources on surfaces (Figure 3). They can be obtained from generic walls by not indicating temperature sources T0 and T1 or they can be described by specific files, such as `walls_in.csv`.

Figure 2a shows a wall of type 1 (described in `wall_types.csv`) with only one layer which has one mesh.

Figure 2b shows a wall of type 0 (described in `wall_types.csv`) with two layers (concrete and insulation). 

![in_wall](./bldg_wall2TC/walls_out.svg)
> Figure 2. Outdoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [5]:
walls = pd.read_csv(folder_path + '/walls_in.csv')
walls

,ID,type,Area,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


In [6]:
# Thermal circuits from data & type files of walls
TCd_out = pd_dm4bem.wall2TC(wall_types, walls, prefix="o")

# print('TCd_out')
# for key in TCd_out.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_out[key])

### Indoor walls

Indoor walls do not have temperature sources on surfaces (Figure 3). They can be obtained from generic walls by not indicating temperature sources T0 and T1 or they can be described by specific files, such as `walls_in.csv`.

Figure 3a shows a wall of type 1 (described in `wall_types.csv`) with two layers (concrete and insulation); the insulation layer is discretized in two meshes.

![in_wall](./bldg_wall2TC/walls_in.svg)
> Figure 3. Indoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [7]:
walls = pd.read_csv(folder_path + '/walls_in.csv')
walls

,ID,type,Area,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


In [8]:
# Thermal circuits from data & type files of walls
TCd_in = pd_dm4bem.wall2TC(wall_types, walls, prefix="i")

# print('TCd_in')
# for key in TCd_in.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_in[key])